# Model 1 - custom MCMC

This example builds upon the previous notebook, so we'll be more terse about some of the stuff already covered, and focus on how to do posterior estimation using MCMC.

## Define the posterior density function

In [3]:
using StatsFuns

# define Φ() as the cumulative normal distribution function
Φ(x) = normcdf.(0, 1, x)

# define the deterministic node PC, corresponds to Equation 2 in Vincent (2015)
PC(Δμ, σ2::Number) = Φ(Δμ/sqrt(2σ2));

log_prior(σ2) = log(1/1000)

function log_likelihood(σ2::Number, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Integer)
    log_likelihood = sum(binomlogpdf.(T, PC(Δμ, σ2), k))
end

log_posterior(σ2, Δμ, k, T::Integer) = log_prior(σ2) + log_likelihood(σ2, Δμ, k, T)

posterior_density(σ2, Δμ, k, T) = exp(log_posterior(σ2, Δμ, k, T::Integer))

posterior_density (generic function with 1 method)

## Define our data

In [4]:
data = Dict()
data["T"] = 100
data["k"] = [50, 51, 57, 55, 63, 62, 82, 94, 99, 100]
data["Δμ"] = [0.0100, 0.0215, 0.0464, 0.1000, 0.2154, 0.4642, 1.0000, 2.1544, 4.6416, 10.0000];

## Define the MH Algorithm

In [41]:
using Distributions

function mhAlgorithm(pdf, Δμ::Array{Float64,1}, k::Array{Int64,1}, T::Number, θ₀::Number, nSamples=10^5, proposalσ=0.1)
    
    accepted = 0
    samples = zeros(Float64, nSamples,1)
    samples[1] = θ₀

    for n = 2:nSamples
        old_sample = samples[n-1]
        old_posterior = pdf(old_sample, Δμ, k, T)

        # suggest a new sample, but θ can't be less than or equal to zero
        new_sample = -Inf
        while new_sample ≤ 0
            new_sample = rand(Normal(old_sample,proposalσ))
        end
        new_posterior = pdf(new_sample, Δμ, k, T)

        # maybe accept new sample
        if new_posterior > old_posterior
            samples[n] = new_sample
            accepted += 1
        else
            u = rand(Normal(0,1))
            if u < new_posterior/old_posterior
                samples[n] = new_sample
                accepted += 1
            else
                samples[n] = old_sample
            end
        end

        if rem(n,5000)==0
            println("$n of $nSamples")
        end
    end

    acceptance_rate = accepted / nSamples;
    println("Acceptance rate: $(acceptance_rate*100) %")
    return samples
end

mhAlgorithm (generic function with 3 methods)

# Generate samples using the MH algorithm

In [43]:
@time samples = mhAlgorithm(posterior_density, data["Δμ"], data["k"], data["T"], 2);

5000 of 100000
10000 of 100000
15000 of 100000
20000 of 100000
25000 of 100000
30000 of 100000
35000 of 100000
40000 of 100000
45000 of 100000
50000 of 100000
55000 of 100000
60000 of 100000
65000 of 100000
70000 of 100000
75000 of 100000
80000 of 100000
85000 of 100000
90000 of 100000
95000 of 100000
100000 of 100000
Acceptance rate: 85.355 %
  0.211285 seconds (602.46 k allocations: 92.418 MiB, 2.87% gc time)


In [45]:
using Plots
plot(samples, legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20000 
 
 
 40000 
 
 
 60000 
 
 
 80000 
 
 
 100000 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
<polyline clip-path="url(#clip2402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,167.902 32.2276,180.271 32.2331,180.271 32.2385,182.034 32.244,182.034 32.2495,197.943 32.255,197.957 32.2605,185.176 32.266,206.807 32.2714,206.536 
 32.2769,198.325 32.2824,183.173 32.2879,190.241 32.2934,208.307 32.2988,208.678 32.3043,219.594 32.3098,207.488 32.3153,192.054 32.3208,192.054 32.3262,194.35 
 32.3317,191.208 32.3372,184.236 32.3427,175.651 32.3482,177.788 32.3536,168.576 32.3591,189.556 32.3646,212.373 32.3701,187.699 32.3756,193.535 32.3811,202.229 
 32.3865,205.646 32.392,200.51 32.3975,195.268 32.403,172.204 32.4085,179.073 32.4139,198.655 32.4194,198.655 32.4249,203.878 32.4304,199.723 32.4359,210.253 
 32.4413,219.877 32.4468,211.592 32.4523,214.887 32.4578,215.791 32.4633,214.39 32.4687,223.075 32.4742,223.853 32.4797,228.738 32.4852,209.726 32.4907,213.192 
 32.4962,217.136 32.5016,212.137 32.5071,212.223 32.5126,221.052 32.5181,216.141 32.5236,217.089 32.529,208.978 32.5345,208.978 32.54,219.082 32.5455,236.414 
 32.551,230.838 32.5564,230.838 32.5619,234.283 32.5674,233.19 32.5729,232.454 32.5784,248.663 32.5839,250.185 32.5893,264.842 32.5948,258.83 32.6003,267.869 
 32.6058,268.973 32.6113,268.973 32.6167,272.558 32.6222,265.722 32.6277,264.73 32.6332,283.194 32.6387,281.535 32.6441,270.284 32.6496,270.511 32.6551,270.511 
 32.6606,275.503 32.6661,261.268 32.6715,259.319 32.677,251.179 32.6825,251.179 32.688,244.036 32.6935,254.514 32.699,245.413 32.7044,267.52 32.7099,273.084 
 32.7154,273.912 32.7209,257.734 32.7264,253.737 32.7318,269.319 32.7373,262.317 32.7428,267.119 32.7483,281.135 32.7538,280.208 32.7592,270.502 32.7647,251.03 
 32.7702,252.031 32.7757,244.264 32.7812,244.264 32.7866,244.658 32.7921,242.24 32.7976,250.336 32.8031,257.718 32.8086,251.585 32.8141,260.355 32.8195,271.849 
 32.825,271.849 32.8305,271.849 32.836,274.326 32.8415,265.035 32.8469,276.761 32.8524,265.53 32.8579,254.67 32.8634,254.801 32.8689,254.555 32.8743,254.882 
 32.8798,245.691 32.8853,245.691 32.8908,259.147 32.8963,259.657 32.9017,281.855 32.9072,283.533 32.9127,283.533 32.9182,280.969 32.9237,280.969 32.9292,278.512 
 32.9346,265.806 32.9401,270.264 32.9456,280.803 32.9511,277.49 32.9566,276.989 32.962,296.739 32.9675,291.038 32.973,291.038 32.9785,272.515 32.984,288.845 
 32.9894,282.072 32.9949,272.833 33.0004,272.833 33.0059,290.67 33.0114,290.67 33.0168,299.748 33.0223,287.378 33.0278,289.589 33.0333,273.502 33.0388,275.507 
 33.0443,273.911 33.0497,273.911 33.0552,271.43 33.0607,264.674 33.0662,262.87 33.0717,264.867 33.0771,244.583 33.0826,237.876 33.0881,224.824 33.0936,241.669 
 33.0991,255.527 33.1045,245.169 33.11,246.237 33.1155,248.73 33.121,240.295 33.1265,240.295 33.1319,237.09 33.1374,225.328 33.1429,235.837 33.1484,242.589 
 33.1539,237.674 33.1594,225.408 33.1648,225.408 33.1703,234.002 33.1758,233.583 33.1813,251.848 33.1868,254.28 33.1922,273.372 33.1977,273.188 33.2032,273.013 
 33.2087,260.474 33.2142,266.955 33.2196,269.923 33.2251,270.186 33.2306,275.522 33.2361,279.669 33.2416,274.698 33.247,274.698 33.2525,274.698 33.258,260.789 
 33.2635,252.219 33.269,240.09 33.2745,249.47 33.2799,264.528 33.2854,270.26 33.2909,272.154 33.2964,272.628 33.3019,258.441 33.3073,259.795 33.3128,256.847 
 33.3183,256.847 33.3238,272.954 33.3293,253.721 33.3347,268.068 33.3402,244.167 33.3457,234.143 33.3512,219.902 33.3567,230.758 33.3622,209.614 33.3676,217.204 
 33.3731,229.465 33.3786,207.062 33.3841,197.069 33.3896,194.238 33.395,196.698 33.4005,199.374 33.406,188.017 33.4115,187.613 33.417,188.333 33.4224,198.012 
 33.4279,185.651 33.4334,194.92 33.4389,196.147 33.4444,219.405 33.4498,212.47 33.4553,210.214 33.4608,226.747 33.4663,230.372

## References
Vincent, B. T. (2015). A tutorial on Bayesian models of perception. Journal of Mathematical Psychology, 66, 103–114. http://doi.org/10.1016/j.jmp.2015.02.001